# Revisiting the Strategies in Federated Learning

As mentioned in the previous unit, Strategies are at the core of federated learning. They determine how clients are selected, which updates are used, and how the new changes are aggregated.

In this unit, we will focus on custom Strategies. To begin, we need to set up the environment for this notebook's development.

### Exercise
As you are familiar with one of the deep learning frameworks, you can implement the following part based on your preference, either PyTorch, Tensorflow, or JAX.


In [1]:
from typing import List, Dict, Tuple, Union, Optional
import numpy as np
import tensorflow as tf
import flwr as fl

##############
#    DATA    #
##############

#Load the CIFAR-10 in different subsets for the training and test as it has been in the previous unit
def load_dataset():
    # Load data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    # Normalize data
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    # Pick only 10000 samples for training and 1000 for testing
    x_train, y_train = x_train[:10_000], y_train[:10_000]
    x_test, y_test = x_test[:1000], y_test[:1000]
    
    return (x_train, y_train), (x_test, y_test)

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def split_index(a, n):
    s = np.array_split(np.arange(len(a)), n)
    return s

def split_dataset(train_dataset, test_dataset, n_clients):
    (x_train, y_train), (x_test, y_test) = train_dataset, test_dataset

    # Randomize the datasets
    x_train, y_train = unison_shuffled_copies(x_train, y_train)
    x_test, y_test = unison_shuffled_copies(x_test, y_test)

    # Split training set into 10 partitions to simulate the individual dataset
    train_index = split_index(x_train, n_clients)
    test_index = split_index(x_test, n_clients)

    # Split each partition
    train_ds = []
    val_ds = []
    test_ds = []
    for cid in range(n_clients):
        val_size = len(train_index[cid]) // 10
        train_input_data, train_output_data = x_train[train_index[cid]], y_train[train_index[cid]]
        val_input_data, val_output_data = train_input_data[:val_size], train_output_data[:val_size]
        train_input_data, train_output_data = train_input_data[val_size:], train_output_data[val_size:]
        train_dataset = (train_input_data, train_output_data)
        val_dataset = (val_input_data, val_output_data)
        test_dataset = (x_test[test_index[cid]], y_test[test_index[cid]])
        train_ds.append(train_dataset)
        val_ds.append(val_dataset)
        test_ds.append(test_dataset)
    return train_ds, val_ds, test_ds

n_clients = 10
train_dataset, test_dataset = load_dataset()
train_datasets, val_datasets, test_datasets = split_dataset(train_dataset, test_dataset, n_clients)

###############
#    MODEL    #
###############

# Define a simple model using TensorFlow
def generate_ann():
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )

    model.compile(
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"],
    )
    return model

###################
#    FL CLIENT    #
###################

def get_parameters(model) -> List[np.ndarray]:
    return model.get_weights()


def set_parameters(model, parameters: List[np.ndarray]):
    model.set_weights(parameters)
    return model

def train(model, train_dataset, epochs: int):
    model.fit(train_dataset[0], train_dataset[1],
            epochs=epochs, batch_size=32, steps_per_epoch=3)
    return model

def test(model, test_dataset):
    loss, accuracy = model.evaluate(test_dataset[0], test_dataset[1])
    return loss, accuracy


# Define the Client class to hold the functions get_parameters, fit and evaluate
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        self.net = set_parameters(self.net, parameters)
        self.net = train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        self.net = set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"[Client {self.cid}] loss:{loss}, accuracy:{accuracy}")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


# Also you can define the generation function of CLients for a stateless version
def client_fn(cid) -> FlowerClient:
    # Create the model
    net = generate_ann()
    #Take the appropiate part of the dataset
    trainloader = train_datasets[int(cid)]
    valloader = val_datasets[int(cid)]
    #Create and return the Client
    return FlowerClient(cid, net, trainloader, valloader)

2023-03-09 19:47:33.129068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Considering the previous code, the model developed has several possibilities for implementing the Strategy object such as `FedAvg` or `FedAdagrad`,  as seen in the  previous Unit. For example, the following code should create a strategy. 

In [2]:
# Create an instance of the model and get the parameters
params = get_parameters(generate_ann())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3, # percentage of clients used for training
    fraction_evaluate=0.3, # percentage of clients used for evaluation
    min_fit_clients=3, # Never sample less than 'min_fit_clients' clients for training
    min_evaluate_clients=3, #Never sample less than 'min_evaluate_clients' clients for evaluation
    min_available_clients=10, # Wait until 'min_available_clients' clients are available
    initial_parameters=fl.common.ndarrays_to_parameters(params), # Initial model parameters
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

2023-03-09 18:33:15.778654: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO flwr 2023-03-09 18:33:16,933 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-03-09 18:33:26,583	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-03-09 18:33:31,449 | app.py:179 | Flower VCE: Ray initialized with resources: {'CPU': 4.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 1033074278.0, 'memory': 2066148558.0}
INFO flwr 2023-03-09 18:33:31,454 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-09 18:33:31,455 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-03-09 18:33:31,471 | serve

(launch_and_fit pid=25947) [Client 1] fit, config: {}
(launch_and_fit pid=25945) [Client 3] fit, config: {}
(launch_and_fit pid=25946) [Client 0] fit, config: {}


DEBUG flwr 2023-03-09 18:33:54,880 | server.py:229 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-03-09 18:33:54,910 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-03-09 18:33:54,912 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)


3/3 [==============================] - 2s 14ms/step - loss: 2.5331 - accuracy: 0.1042
(launch_and_evaluate pid=25946) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=25945) [Client 8] evaluate, config: {}
4/4 [==============================] - 1s 24ms/step - loss: 2.2507 - accuracy: 0.1500
(launch_and_evaluate pid=25946) [Client 1] loss:2.2506637573242188, accuracy:0.15000000596046448
4/4 [==============================] - 1s 14ms/step - loss: 2.3590 - accuracy: 0.1100
(launch_and_evaluate pid=25945) [Client 8] loss:2.3590002059936523, accuracy:0.10999999940395355
(launch_and_evaluate pid=25946) [Client 2] evaluate, config: {}


DEBUG flwr 2023-03-09 18:34:04,069 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-03-09 18:34:04,070 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-03-09 18:34:04,073 | server.py:215 | fit_round 2: strategy sampled 3 clients (out of 10)


...

3/3 [==============================] - 1s 6ms/step - loss: 2.4127 - accuracy: 0.0521
(launch_and_evaluate pid=25945) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=25947) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=25946) [Client 7] evaluate, config: {}


DEBUG flwr 2023-03-09 18:34:39,852 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-03-09 18:34:39,853 | server.py:144 | FL finished in 68.34607747301925
INFO flwr 2023-03-09 18:34:39,865 | app.py:202 | app_fit: losses_distributed [(1, 2.2989168961842856), (2, 2.295743147532145), (3, 2.293851375579834)]
INFO flwr 2023-03-09 18:34:39,868 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-03-09 18:34:39,869 | app.py:204 | app_fit: losses_centralized []
INFO flwr 2023-03-09 18:34:39,869 | app.py:205 | app_fit: metrics_centralized {}


4/4 [==============================] - 1s 3ms/step - loss: 2.2768 - accuracy: 0.1100
(launch_and_evaluate pid=25947) [Client 3] loss:2.2767512798309326, accuracy:0.10999999940395355
4/4 [==============================] - 1s 3ms/step - loss: 2.3008 - accuracy: 0.1000
(launch_and_evaluate pid=25945) [Client 9] loss:2.300809860229492, accuracy:0.10000000149011612


History (loss, distributed):
	round 1: 2.2989168961842856
	round 2: 2.295743147532145
	round 3: 2.293851375579834

4/4 [==============================] - 1s 3ms/step - loss: 2.3040 - accuracy: 0.0800
(launch_and_evaluate pid=25946) [Client 7] loss:2.303992986679077, accuracy:0.07999999821186066


It may be worth mentioning that Flower, by default, initializes the global model by making a call to one random client before distributing it to the remaining clients. However, sometimes more control is required, such as when performing fine-tuning. In such situations, we use server-side initialization, and the `initial_parameters` parameter will hold the initial version of the model for all clients. It is important to note that this parameter must be a serialization of the data, so the utility function `ndarrays_to_parameters` can be quite handy in this case.

Now, let's move on to customizing the type of evaluation performed on the models. Broadly speaking, there are two possibilities: server-side evaluation and client-side evaluation.

**Centralized evaluation** (server-side) is similar to traditional machine learning, where the server holds a partition solely for evaluating the aggregated model. This approach reduces communication and is suitable for situations with limited bandwidth. There is no need to send the model to the clients for evaluation, and the entire evaluation dataset is available at all times.

**Federated evaluation** (client-side) is more complex, but it usually represents real-world scenarios more accurately. In this approach, the evaluation dataset is distributed among the clients, which means that we can leverage a larger dataset spread among the resources of the clients. However, this approach comes with a cost. Since we don't have a central dataset, we should be aware that our evaluation dataset can change over consecutive rounds of learning if some clients are not always available. Moreover, the dataset held by each client can also change over consecutive rounds. This can lead to evaluation results that are not stable, so even if we don't change the model, we can see our evaluation results fluctuate over consecutive rounds. Additionally, this approach can significantly increase the number of communications because the models have to be distributed among the clients and retrieved for evaluation.

The previous code snippet is an example of Flower performing Federated evaluations, as it uses the `evaluation` function that is executed on each `Client` and later aggregated after being sent to the server. On the other hand, a Centralized evaluation could be performed with a similar approach, as shown in the following code snippet:


In [3]:
# The `evaluate` function will be by Flower called after every round -> return function to avoid model loading overhead
def get_evaluate_fn(model):
    def evaluate(server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
        dataset = test_dataset # Check if this name is the appropriate according to the developed exercise. It should be a Tuple
        set_parameters(model, parameters)  # Update model with the latest parameters
        loss, accuracy = test(model, dataset)
        print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
        return loss, {"accuracy": accuracy}
    return evaluate

# Model to initialize the global model and allow evaluation
model = generate_ann() 

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=10,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(model)),
    evaluate_fn=get_evaluate_fn(model),  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

INFO flwr 2023-03-09 18:41:34,228 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-03-09 18:41:46,616	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-03-09 18:41:51,492 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 2440962048.0, 'CPU': 4.0, 'object_store_memory': 1220481024.0, 'node:127.0.0.1': 1.0}
INFO flwr 2023-03-09 18:41:51,497 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-09 18:41:51,498 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-03-09 18:41:51,499 | server.py:88 | Evaluating initial parameters


32/32 [==============================] - 1s 3ms/step - loss: 2.3759 - accuracy: 0.1010


INFO flwr 2023-03-09 18:41:53,026 | server.py:91 | initial parameters (loss, other metrics): 2.3758909702301025, {'accuracy': 0.10100000351667404}
INFO flwr 2023-03-09 18:41:53,027 | server.py:101 | FL starting
DEBUG flwr 2023-03-09 18:41:53,027 | server.py:215 | fit_round 1: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 2.3758909702301025 / accuracy 0.10100000351667404


(launch_and_fit pid=26192) 2023-03-09 18:41:56.120036: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(launch_and_fit pid=26192) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(launch_and_fit pid=26194) 2023-03-09 18:41:56.120726: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(launch_and_fit pid=26194) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(launch_and_fit pid=26191) 2023-03-09 18:41:56.120036: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the foll

...

INFO flwr 2023-03-09 18:42:34,379 | server.py:116 | fit progress: (3, 2.3214502334594727, {'accuracy': 0.11299999803304672}, 41.35100747901015)
DEBUG flwr 2023-03-09 18:42:34,382 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 2.3214502334594727 / accuracy 0.11299999803304672
(launch_and_evaluate pid=26191) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=26192) [Client 4] evaluate, config: {}
4/4 [==============================] - 1s 5ms/step - loss: 2.3150 - accuracy: 0.1100
(launch_and_evaluate pid=26192) [Client 4] loss:2.314965009689331, accuracy:0.10999999940395355
4/4 [==============================] - 1s 5ms/step - loss: 2.3455 - accuracy: 0.1400
(launch_and_evaluate pid=26191) [Client 3] loss:2.3455028533935547, accuracy:0.14000000059604645
(launch_and_evaluate pid=26192) [Client 2] evaluate, config: {}


DEBUG flwr 2023-03-09 18:42:38,464 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-03-09 18:42:38,465 | server.py:144 | FL finished in 45.43772968597477
INFO flwr 2023-03-09 18:42:38,467 | app.py:202 | app_fit: losses_distributed [(1, 2.3243539333343506), (2, 2.2827131748199463), (3, 2.321775436401367)]
INFO flwr 2023-03-09 18:42:38,468 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-03-09 18:42:38,469 | app.py:204 | app_fit: losses_centralized [(0, 2.3758909702301025), (1, 2.325075387954712), (2, 2.2725231647491455), (3, 2.3214502334594727)]
INFO flwr 2023-03-09 18:42:38,473 | app.py:205 | app_fit: metrics_centralized {'accuracy': [(0, 0.10100000351667404), (1, 0.10899999737739563), (2, 0.1120000034570694), (3, 0.11299999803304672)]}


4/4 [==============================] - 0s 2ms/step - loss: 2.3049 - accuracy: 0.1400
(launch_and_evaluate pid=26192) [Client 2] loss:2.304858446121216, accuracy:0.14000000059604645


History (loss, distributed):
	round 1: 2.3243539333343506
	round 2: 2.2827131748199463
	round 3: 2.321775436401367
History (loss, centralized):
	round 0: 2.3758909702301025
	round 1: 2.325075387954712
	round 2: 2.2725231647491455
	round 3: 2.3214502334594727
History (metrics, centralized):
{'accuracy': [(0, 0.10100000351667404), (1, 0.10899999737739563), (2, 0.1120000034570694), (3, 0.11299999803304672)]}

Additionally, it is possible to implement a custom strategy from scratch by implementing the necessary methods and extending `flwr.server.strategy.Strategy`. The required methods for a custom strategy are as follows:
* `num_fit_clients`: returns the number of clients to be selected for the next round of training.
* `num_rounds`: returns the number of rounds of training to perform.
* `on_fit`: called when a client has completed training and returned its updated model. This method should update the global model based on the returned model.
* `on_evaluate`: called when a client has completed an evaluation and returned its evaluation result. This method should aggregate the evaluation results.


You can see an schema of the methods and an example in the following [link](https://flower.dev/docs/tutorial/Flower-3-Building-a-Strategy-PyTorch.html)

# Challenges for Federated Learning

While federated learning can solve problems that traditional centralized machine learning struggles with, such as privacy and reduced hardware requirements, it also presents its own challenges. In this section, we will cover some of these challenges, including the non-IID (independent and identically distributed) nature of the data, the heterogeneous nature of devices, and the limited communication bandwidth.

## Non-IID data
The assumption of independence and identical distribution, or i.i.d., is commonly made in machine learning and statistical analysis. This means that each data point is independent of all other data points, and that the distribution of the data is the same across all data points.

Non-i.i.d. data, on the other hand, violates one or both of these assumptions. This can occur for a variety of reasons. For example, data may be collected in a way that introduces dependencies between data points, such as when data is collected over time or in a specific order. Additionally, the distribution of the data may vary across different subgroups or regions, making it non-i.i.d. Non-i.i.d. data is a common challenge in federated learning because the data is distributed across many devices, and each device may have a different distribution of data due to variations in data collection methods or data sources. As a result, traditional machine learning algorithms may not perform well on non-i.i.d. data.

In a non-IID data problem (see Figure 1(a)), "non-IIDness" (see Figure 1(c)) refers to the presence of couplings (such as co-occurrence, neighborhood, dependency, linkage, correlation, and causality) and heterogeneities within and between two or more aspects, such as entities, entity classes, entity properties (variables), processes, facts, and states of affairs, or other types of entities or properties (such as learners and learned results) that appear or are produced prior to, during, and after a target process (such as a learning task). Conversely, IIDness ignores or simplifies these relationships, as shown in Figure 1(b).

![Diagram with IID and non-IID data](https://datasciences.org/wp-content/themes/dslabNew/images/datasciences/IIDness.png)
Credit: [Source of the image](https://datasciences.org/non-iid-learning/)

Non-i.i.d. data can be more challenging to work with than i.i.d. data because standard statistical assumptions and techniques may not be applicable. Therefore, special techniques may need to be employed to analyze non-i.i.d. data, which may include techniques that take into account the dependencies between data points or the varying data distributions.

In this context, non-i.i.d. data refers to the fact that the data on each device may differ in terms of distribution, characteristics, and relevance to the task at hand. For instance, the data on one device may comprise mainly images of dogs, while the data on another device may consist mainly of images of cats. This can pose a challenge in training a model that performs well on all the devices because the data on each device can vary significantly from the data on the other devices.

To address non-i.i.d. data in federated learning, special techniques are often employed to weigh the contributions of each device's data to the overall model, or to adjust the model's parameters in a way that considers the differences in the data. Furthermore, techniques such as data augmentation and transfer learning could help to generalize the model beyond the device's data.

When discussing Flower, the approach to addressing this problem would involve implementing a custom strategy, similar to the following example, that uses a custom aggregation of the results.


In [4]:
from flwr.common import EvaluateRes, FitRes, Scalar
from flwr.server.client_proxy import ClientProxy

class AggregateCustomMetricStrategy(fl.server.strategy.FedAvg):
    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation accuracy using weighted average."""

        if not results:
            return None, {}

        # Call aggregate_evaluate from base class (FedAvg) to aggregate loss and metrics
        aggregated_loss, aggregated_metrics = super().aggregate_evaluate(server_round, results, failures)

        # Weigh accuracy of each client by number of examples used
        accuracies = [r.metrics["accuracy"] * r.num_examples for _, r in results]
        examples = [r.num_examples for _, r in results]

        # Aggregate and print custom metric
        aggregated_accuracy = sum(accuracies) / sum(examples)
        print(f"Round {server_round} accuracy aggregated from client results: {aggregated_accuracy}")

        # Return aggregated loss and metrics (i.e., aggregated accuracy)
        return aggregated_loss, {"accuracy": aggregated_accuracy}

# Model to initialize the global model and allow evaluation
model = generate_ann()

# Create strategy and run server
strategy = AggregateCustomMetricStrategy(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=10,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(model)),
)


fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)


INFO flwr 2023-03-09 19:22:20,364 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-03-09 19:22:28,913	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-03-09 19:22:30,670 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 2454972827.0, 'object_store_memory': 1227486412.0, 'CPU': 4.0, 'node:127.0.0.1': 1.0}
INFO flwr 2023-03-09 19:22:30,674 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-09 19:22:30,676 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-03-09 19:22:30,678 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-03-09 19:22:30,679 | server.py:101 | FL starting
DEBUG flwr 2023-03-09 19:22:30,680 | server.py:215 | fit_round 1: strategy sampled 3 clients (out of 10)
(launch_and_fit pid=27570) 2023-03-09 19:22:33.617484: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is

(launch_and_fit pid=27571) [Client 9] fit, config: {}
(launch_and_fit pid=27568) [Client 6] fit, config: {}
(launch_and_fit pid=27570) [Client 5] fit, config: {}


DEBUG flwr 2023-03-09 19:22:45,156 | server.py:229 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-03-09 19:22:45,176 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-03-09 19:22:45,178 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)


3/3 [==============================] - 1s 12ms/step - loss: 2.3147 - accuracy: 0.1354
(launch_and_evaluate pid=27570) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=27571) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=27568) [Client 0] evaluate, config: {}


DEBUG flwr 2023-03-09 19:22:48,826 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-03-09 19:22:48,827 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-03-09 19:22:48,828 | server.py:215 | fit_round 2: strategy sampled 3 clients (out of 10)


...

3/3 [==============================] - 2s 3ms/step - loss: 2.3477 - accuracy: 0.1250
(launch_and_evaluate pid=27568) [Client 7] evaluate, config: {}
4/4 [==============================] - 0s 3ms/step - loss: 2.3862 - accuracy: 0.1100
(launch_and_evaluate pid=27568) [Client 7] loss:2.386219024658203, accuracy:0.10999999940395355
(launch_and_evaluate pid=27571) [Client 0] evaluate, config: {}
4/4 [==============================] - 0s 3ms/step - loss: 2.2786 - accuracy: 0.1200
(launch_and_evaluate pid=27571) [Client 0] loss:2.278578996658325, accuracy:0.11999999731779099
(launch_and_evaluate pid=27568) [Client 5] evaluate, config: {}


DEBUG flwr 2023-03-09 19:23:09,938 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-03-09 19:23:09,939 | server.py:144 | FL finished in 39.25867025798652
INFO flwr 2023-03-09 19:23:09,942 | app.py:202 | app_fit: losses_distributed [(1, 2.234875520070394), (2, 2.2884956995646157), (3, 2.3180171648661294)]
INFO flwr 2023-03-09 19:23:09,944 | app.py:203 | app_fit: metrics_distributed {'accuracy': [(1, 0.16000000139077505), (2, 0.14333333323399225), (3, 0.11999999731779099)]}
INFO flwr 2023-03-09 19:23:09,945 | app.py:204 | app_fit: losses_centralized []
INFO flwr 2023-03-09 19:23:09,946 | app.py:205 | app_fit: metrics_centralized {}


Round 3 accuracy aggregated from client results: 0.11999999731779099


History (loss, distributed):
	round 1: 2.234875520070394
	round 2: 2.2884956995646157
	round 3: 2.3180171648661294
History (metrics, distributed):
{'accuracy': [(1, 0.16000000139077505), (2, 0.14333333323399225), (3, 0.11999999731779099)]}

4/4 [==============================] - 0s 2ms/step - loss: 2.2893 - accuracy: 0.1300
(launch_and_evaluate pid=27568) [Client 5] loss:2.2892534732818604, accuracy:0.12999999523162842


## Heterogeneity of the devices

The heterogeneity of the devices in the network, which means they may have different hardware and software configurations and may be running different versions of the operating system, is one of the problems of federated learning. This can lead to several problems, including:

* Inefficient communication: Different devices may have varying network speeds and bandwidth, which can make it difficult to transmit model updates between devices in a timely manner.

* Incompatible updates: If different devices are running different versions of the operating systems, they may not be able to exchange model updates due to compatibility issues.

* Data heterogeneity: The data on different devices may differ in terms of quality, quantity, and format, making it challenging to train a model that generalizes well across all devices.



To mitigate the impact of heterogeneous devices in federated learning, researchers are developing techniques such as device-aware aggregation algorithms and communication optimization. These techniques aim to address issues such as inefficient communication and incompatible updates resulting from differences in network speeds, bandwidth, operating system versions, and data heterogeneity across the devices.


Consider a network of five devices (A, B, C, D, and E) that are participating in federated learning to train a global model. Each device has its own data and trains a local model on that data. The local models are then transmitted back to a central server, where they are aggregated and used to update the global model.


In the above scenario, the participating devices (A, B, C, D, and E) in the federated learning network are heterogeneous in nature, meaning they possess different hardware and software configurations. For instance, Device A and Device B may be running distinct versions of the operating system, and Device C may have a slower network connection in comparison to the other devices.


This heterogeneity in the devices can create challenges in the federated learning process. For instance, Device A may face difficulty sending its local model update to the server because of compatibility issues with Device B, and Device C may experience a slower transmission due to its slower network connection.


To overcome the challenges posed by heterogeneous devices in federated learning, researchers are developing techniques to mitigate their impact. These techniques may include device-aware aggregation algorithms, which take into account the different hardware and software configurations of the devices, and communication optimization techniques such as data compression and intelligent routing. By adapting the way that data is aggregated and transmitted, these techniques can help to ensure that all devices are able to contribute effectively to the global model, regardless of their individual characteristics.



It is also worth mentioning that a local configuration can be provided to the `Clients` by means of the `config` parameter of the function in the `FlowerClient`. This parameter is a Python `Dict` which holds values that can be used internally for different purposes, such as limiting the number of epochs on certain clients or establishing the number of rounds.


The modification for the strategy in this case would require the use of parameter `on_fit_config` to indicate the function to retrieve the correct configuration.


```python
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,
    }
    return config

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=10,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(model)),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

```

However, sometimes limiting the number of rounds or the number of epochs for each client is not enough, especially when the number of clients is too large to handle. In such cases, it may be necessary to reduce the number of clients used for training and evaluation. For instance, consider a scenario where there are 1000 clients, each with only 50 samples for training and 10 for evaluation. Although the amount of data in each client is limited, the communication overhead can still be overwhelming. In such cases, it is better to train for a longer time with a smaller number of clients in each round.

In [3]:
# Be careful with this cell. It's a time consuming process
n_clients = 40

train_datasets, val_datasets, test_datasets = split_dataset(train_dataset, test_dataset, n_clients)

# Also you can define the generation function of CLients for a stateless version
def client_fn(cid) -> FlowerClient:
    # Create the model
    net = generate_ann()
    #Take the appropiate part of the dataset
    trainloader = train_datasets[int(cid)]
    valloader = val_datasets[int(cid)]
    #Create and return the Client
    return FlowerClient(cid, net, trainloader, valloader)

# Model to initialize the global model and allow evaluation
model = generate_ann()

def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=n_clients,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(model)),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=n_clients,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

2023-03-09 19:48:33.287257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO flwr 2023-03-09 19:48:33,420 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-03-09 19:48:38,304	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-03-09 19:48:41,124 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 1054105190.0, 'node:127.0.0.1': 1.0, 'CPU': 4.0, 'memory': 2108210382.0}
INFO flwr 2023-03-09 19:48:41,126 | server.py:86 | Initializing global parameters
INFO flwr 2023-03-09 19:48:41,128 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-03-09 19:48:41,136 | serve

(launch_and_fit pid=28563) [Client 18] fit, config: {'server_round': 1, 'local_epochs': 3}


(launch_and_fit pid=28565) 2023-03-09 19:49:09.396017: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(launch_and_fit pid=28565) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(launch_and_fit pid=28564) [Client 29] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=28565) [Client 6] fit, config: {'server_round': 1, 'local_epochs': 3}
3/3 [==============================] - 2s 15ms/step - loss: 2.4959 - accuracy: 0.1354


(launch_and_fit pid=28562) 2023-03-09 19:49:13.784267: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(launch_and_fit pid=28562) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


...

1/1 [==============================] - 2s 2s/step - loss: 2.3194 - accuracy: 0.1200
(launch_and_evaluate pid=28562) [Client 4] loss:2.319409132003784, accuracy:0.11999999731779099
1/1 [==============================] - 1s 1s/step - loss: 2.2899 - accuracy: 0.1200
(launch_and_evaluate pid=28565) [Client 22] loss:2.287884473800659, accuracy:0.03999999910593033


DEBUG flwr 2023-03-09 19:54:35,247 | server.py:179 | evaluate_round 3 received 17 results and 23 failures
INFO flwr 2023-03-09 19:54:35,253 | server.py:144 | FL finished in 354.10427461203653
INFO flwr 2023-03-09 19:54:35,271 | app.py:202 | app_fit: losses_distributed [(1, 2.3619590520858766), (2, 2.3345150724053383), (3, 2.283243277493645)]
INFO flwr 2023-03-09 19:54:35,274 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-03-09 19:54:35,277 | app.py:204 | app_fit: losses_centralized []
INFO flwr 2023-03-09 19:54:35,278 | app.py:205 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=28563) [Client 1] loss:2.289893865585327, accuracy:0.11999999731779099


History (loss, distributed):
	round 1: 2.3619590520858766
	round 2: 2.3345150724053383
	round 3: 2.283243277493645

(raylet) [2023-03-09 19:54:44,094 E 28559 4984833] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-03-09_19-48-33_583684_28501 is over 95% full, available space: 4048863232; capacity: 121018208256. Object creation will fail if spilling is required.


In addition to the techniques mentioned earlier, federated transfer learning, secure aggregation, and data augmentation are other approaches that can help in the scaling of the federated learning system. The limitation of resources, including bandwidth, storage, and computation power, is one of the main challenges of federated learning.